In [ ]:
from path_guiding import QuadTree, MapSphereToTree

In [ ]:
import numpy as np
from matplotlib import pyplot
import matplotlib
import itertools

In [ ]:
#%matplotlib notebook

In [ ]:
def compute_leaf_mask(qt):
    return np.asarray([ qt.IsLeaf(i) for i in range(qt.NumNodes()) ])

def plot_quadtree(ax, qt, weights=None):
    quads = qt.GenerateQuads()
    if weights is None:
        for xmin, ymin, xmax, ymax in quads:
            ax.add_artist(pyplot.Rectangle((xmin,ymin),xmax-xmin, ymax-ymin, fill=False))
    else:
        mask = compute_leaf_mask(qt)
        normalization = weights[mask].max()
        weights = weights / normalization
        for (xmin, ymin, xmax, ymax), w, i in zip(quads, weights, itertools.count()):
            if mask[i]:
                ax.add_artist(pyplot.Rectangle((xmin,ymin),xmax-xmin, ymax-ymin, facecolor = matplotlib.cm.coolwarm(w), edgecolor='k'))
        return matplotlib.cm.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=0., vmax=normalization), cmap = matplotlib.cm.coolwarm)
    
def compute_areas(qt):
    xmin, ymin, xmax, ymax = qt.GenerateQuads().T
    return (xmax-xmin)*(ymax-ymin)

def compute_area_centers(qt):
    pts = qt.GenerateQuads()
    return 0.5*(pts[:,[0,1]] + pts[:,[2,3]])

def compute_pdfs_from_node_weights(qt, nws):
    return nws/nws[0]/compute_areas(qt)

def sample_normal(N, loc=[0.,0.], scale = 0.2):
    xs = np.random.normal(loc=loc, scale=scale, size=(N,2))
    ws = np.ones((N,), dtype=np.float32)
    return xs, ws

In [ ]:
fig, (ax1, ax2) = pyplot.subplots(1,2, figsize=(13,6))

xs, ws = sample_normal(1000, loc=[0.25, 0.5], scale = 0.05)
qt, nws = QuadTree.Build(xs, ws, 0.1)

eps = 1.e-7
x, y = np.meshgrid(np.linspace(0.,1., 10), np.linspace(0.,1.-eps,10))
rnd = np.vstack((x.ravel(), y.ravel())).T

treesamples, pdfs = qt.Sample(nws, 100)

pdfs_through_func = qt.Pdf(treesamples, nws)

print (np.abs(pdfs-pdfs_through_func).max())
print (np.where(pdfs <= 0.))

plot_quadtree(ax1, qt)
ax1.scatter(*treesamples.T, c=pdfs, zorder=10, cmap = matplotlib.cm.coolwarm)
ax1.set(xlim=(0.,1.),ylim=(0.,1.))

plot_quadtree(ax2, qt, weights = compute_pdfs_from_node_weights(qt,nws))
ax2.set(xlim=(0.,1.),ylim=(0.,1.))

In [ ]:
# Check the sampling code.

fig, (ax1, ax2) = pyplot.subplots(1,2, figsize=(13,6))

#xs, ws = sample_normal(1000, loc=[0.25, 0.5], scale = 0.05)
xs, ws = sample_normal(1000, loc=[0.5, 0.5], scale = 0.05)
qt, nws = QuadTree.Build(xs, ws, 0.1)

rnd = np.random.uniform(size=(100,2))
treesamples, pdfs = qt.Sample(nws, 100)

pdfs_through_func = qt.Pdf(treesamples, nws)

plot_quadtree(ax1, qt)
m = ax1.scatter(*treesamples.T, c=pdfs, zorder=10, cmap = matplotlib.cm.coolwarm)
pyplot.colorbar(m)
ax1.set(xlim=(0.,1.),ylim=(0.,1.))

m = plot_quadtree(ax2, qt, weights = compute_pdfs_from_node_weights(qt,nws))
pyplot.colorbar(m)
ax2.set(xlim=(0.,1.),ylim=(0.,1.))

# I = int 1 dx = int 1/p p dx = E[1/p]

print (np.abs(pdfs_through_func - pdfs).max())

#print (np.sum(pdf) / rnd.shape[0])
mask = compute_leaf_mask(qt)
print (np.sum(compute_pdfs_from_node_weights(qt,nws)[mask]*compute_areas(qt)[mask]))

In [ ]:
# Plot quad tree which has only one leaf node
fig = pyplot.figure()
ax = fig.add_subplot(111)
qt = QuadTree()
plot_quadtree(ax, qt)
pyplot.show()

In [ ]:
# Build tree from normal distributed samples
xs, ws = sample_normal(1000, loc=[0.7, 0.5], scale = 0.05)
qt, nws = QuadTree.Build(xs, ws, 0.1)
fig, ax = pyplot.subplots(1,1, figsize=(6,6))
pyplot.scatter(*xs.T)
m = plot_quadtree(ax, qt, nws / nws[0])
pyplot.colorbar(m)
ax.set(xlim=(0.,1.),ylim=(0.,1.))

In [ ]:
# Test the adaptation.
# It generates some samples and adapts the tree in a loop.
# Plot each step. Also converts the node weightings to the adapted tree.

# Generate initial tree
xs, ws = sample_normal(1000, loc=[0.7, 0.5], scale = 0.05)
qt, nws = QuadTree.Build(xs, ws, 0.1)

fig, ax1 = pyplot.subplots(1,1, figsize=(6,6))
plot_quadtree(ax1, qt, weights = nws)
pyplot.show()


for _ in range(5):
    fig, (ax1, ax2) = pyplot.subplots(1,2, figsize=(13,6))
    
    # Samples to use for adaptation
    xs, ws = sample_normal(1000, loc=[0.25, 0.5], scale = 0.05)
    
    # Generate weights
    nws = np.zeros(qt.NumNodes(), np.float32)
    qt.PushWeights(nws, xs, ws)

    # Plot
    plot_quadtree(ax1, qt, weights = nws)
    ax1.scatter(*xs.T, color='r', zorder=10)
    ax1.set(xlim=(0.,1.),ylim=(0.,1.))

    # Adapt the tree and plot again
    qt, new_nws = qt.Adapted(nws, 0.1)

    plot_quadtree(ax2, qt, new_nws)
    ax2.set(xlim=(0.,1.),ylim=(0.,1.))

    pyplot.show()

In [ ]:
# Check the Pdf

fig, axes = pyplot.subplots(3,2, figsize=(8,20))
ax1, ax2, ax3 = axes[:,0]
cax1, cax2, cax3 = axes[:,1]

# Generate initial tree
xs, ws = sample_normal(1000, loc=[0.7, 0.5], scale = 0.05)
qt, nws = QuadTree.Build(xs, ws, 0.1)

# Plot the relative node weights
m = plot_quadtree(ax1, qt, weights = nws / nws[0])
ax1.scatter(*xs.T, color='g', zorder=10, marker='x', s=2)
ax1.set(xlim=(0.,1.),ylim=(0.,1.))
fig.colorbar(m, cax=cax1)
 
# Plot the pdf as computed in python
m = plot_quadtree(ax2, qt, weights=compute_pdfs_from_node_weights(qt, nws))
ax2.set(xlim=(0.,1.),ylim=(0.,1.))
fig.colorbar(m, cax=cax2)

# Plot the pdf as computed by the c++ code.
centers = compute_area_centers(qt)
pdfs = qt.Pdf(centers, nws)
m = plot_quadtree(ax3, qt, weights=pdfs)
ax3.set(xlim=(0.,1.),ylim=(0.,1.))
fig.colorbar(m, cax=cax3)

# Check if integral of pdf over unit square amounts to 1
mask = compute_leaf_mask(qt)
print (np.sum(pdfs[mask]*compute_areas(qt)[mask]))

ax1.set_aspect(1.)
ax2.set_aspect(1.)
ax3.set_aspect(1.)

cax2.set_aspect(5.)
cax3.set_aspect(5.)
cax1.set_aspect(5.)

pyplot.tight_layout()